<a href="https://colab.research.google.com/github/gracegm/Complete-Python-3-Bootcamp/blob/master/CLV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I don't look at money as something to solve for.

I think of money as a simple VARIABLE in a greater problem that I'm trying to solve.  

Like any other problem it's really the way that you go about solving it that will determine whether or not that problem will persist.

There is nothing more permanent than a temporary fix.



In [ ]:
!pip install lifetimes

In [ ]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)
# data:  https://github.com/fenago/datasets/raw/main/Online%20Retail.xlsx

import lifetimes
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter # BG/NBD
from lifetimes import GammaGammaFitter # Gamma-Gamma Model
from lifetimes.plotting import plot_frequency_recency_matrix
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_excel("https://github.com/fenago/datasets/raw/main/Online%20Retail.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [ ]:
# Ack extreme outliers for Quanity and UnitPrice
# Let's clean the data (however YOU decide to clean... just make sure it is logical)
df = df[df['Quantity'] > 0 ] # exclude the orders with 0 value
df = df[df['UnitPrice'] > 0] # exclude the Unit Price with 0 value
df = df[~df['InvoiceNo'].str.contains("C",na=False)]  # C indicates the returned orders we don't want them as well

In [ ]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132220
Country             0
dtype: int64

In [ ]:
df.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [ ]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397884 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [ ]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [ ]:
capping_outliers(df,'UnitPrice')
capping_outliers(df,'Quantity')

In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,397884.000000,397884.000000,397884.000000
mean,8.868022,2.675785,15294.423453
std,9.523425,2.275053,1713.141560
min,1.000000,0.420000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,36.000000,8.500000,18287.000000


## RFM
### Create a Recency, Frequency and T(ime) : Monetary DataFrame
### Frequency: Number of repeat purchases the customer has made
### Recency: Delta between furst purchase and latest purchase
### T represent the unit that you want to use (days, weeks, months, etc.). Duration between a customers fiesrt purchase (default is days) and the "end of the study"
### M represents the monetoary value of a given customer

In [ ]:
df['Total_Price'] = df['UnitPrice'] * df['Quantity']

In [ ]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Total Price', 'Total_Price'], dtype='object')

In [ ]:
clv = lifetimes.utils.summary_data_from_transaction_data(df,'CustomerID','InvoiceDate','Total_Price',observation_period_end='2011-12-09')

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
CustomerID,,,,
12346.0,0.0,0.0,325.0,0.000000
12347.0,6.0,365.0,367.0,550.570000
12348.0,3.0,283.0,358.0,116.126667
12349.0,0.0,0.0,18.0,0.000000
12350.0,0.0,0.0,310.0,0.000000


In [ ]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
CustomerID,,,,
12347.0,6.0,365.0,367.0,550.570000
12348.0,3.0,283.0,358.0,116.126667
12352.0,6.0,260.0,296.0,192.840000
12356.0,2.0,303.0,325.0,226.080000
12359.0,3.0,274.0,331.0,1495.650000


# BG/NBD Model

In [ ]:
bgf = BetaGeoFitter(penalizer_coef=0.001)  # This will allow you to predict FUTURE purchase by taking into account F, R, T  penalizer is used to avoid overfitting
bgf.fit(clv['frequency'], clv['recency'], clv['T'])

<lifetimes.BetaGeoFitter: fitted with 1916 subjects, a: 0.00, alpha: 109.98, b: 0.00, r: 2.35>

In [ ]:
t = 180 # 6 month/180 day period
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, clv['frequency'], clv['recency'], clv['T'])
clv.sort_values(by='expected_purc_6_months',ascending=False).head(5)

,frequency,recency,T,monetary_value,expected_purc_6_months
CustomerID,,,,,
14911.0,131.0,372.0,373.0,917.278855,49.698999
12748.0,112.0,373.0,373.0,257.314911,42.617895
17841.0,111.0,372.0,373.0,349.079640,42.245205
15311.0,89.0,373.0,373.0,421.881573,34.046032
14606.0,88.0,372.0,373.0,125.302955,33.673343


# Gamma-Gamma Model

In [ ]:
clv[['frequency','monetary_value']].corr()

,frequency,monetary_value
frequency,1.000000,0.110771
monetary_value,0.110771,1.000000


In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(clv["frequency"],
        clv["monetary_value"])

<lifetimes.GammaGammaFitter: fitted with 1916 subjects, p: 3.79, q: 0.34, v: 3.72>

In [ ]:
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv,six_Months_CLV
CustomerID,,,,,,,
14096.0,16.0,97.0,101.0,3012.454375,15.657745,46062.314700,46062.314700
14911.0,131.0,372.0,373.0,917.278855,49.698999,44093.511057,44093.511057
14646.0,44.0,353.0,354.0,2507.804091,17.982416,43732.700984,43732.700984
14156.0,42.0,362.0,371.0,1366.275476,16.598352,21996.982767,21996.982767
18102.0,25.0,367.0,367.0,2112.843200,10.322125,21214.008259,21214.008259


In [ ]:
clv['Segment'] =  pd.qcut(clv['six_Months_CLV'],4,labels = ['Hibernating','Need Attention', 'LoyalCustomers', 'Champions'])

In [ ]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv,six_Months_CLV,Segment
CustomerID,,,,,,,,
12347.0,6.0,365.0,367.0,550.570000,3.151946,1727.959712,1727.959712,Champions
12348.0,3.0,283.0,358.0,116.126667,2.058657,247.676823,247.676823,Hibernating
12352.0,6.0,260.0,296.0,192.840000,3.703182,712.558776,712.558776,Need Attention
12356.0,2.0,303.0,325.0,226.080000,1.801024,434.111529,434.111529,Hibernating
12359.0,3.0,274.0,331.0,1495.650000,2.184705,3352.235412,3352.235412,Champions


In [ ]:
clv.groupby('Segment').mean()

,frequency,recency,T,monetary_value,expected_purc_6_months,6_monhths_clv,six_Months_CLV
Segment,,,,,,,
Hibernating,3.169102,220.565762,291.824635,148.661593,2.552501,369.465054,369.465054
Need Attention,4.018789,239.442589,282.941545,270.869214,3.004030,763.560023,763.560023
LoyalCustomers,5.682672,241.569937,273.110647,369.673664,3.834613,1276.982217,1276.982217
Champions,11.244259,264.486430,284.411273,636.264425,6.007552,3416.144496,3416.144496
